In [17]:
import pandas as pd

In [18]:
pathDonnees = "../DataSetBrut/SMSSpamCollection.txt"
trainProportion = 0.5

In [19]:
df = pd.read_csv(
    pathDonnees, delimiter="\t", header=None, names=["spam", "text"]
)


In [20]:
sizeData = len(df.index)
trainSize = int(trainProportion*sizeData)
trainDF = df.sample(trainSize)
testDF = df.drop(trainDF.index)


In [21]:
trainDFSpam = trainDF[trainDF["spam"]=="spam"]
trainDFSpam["text"]=trainDFSpam["text"].str.lower()
dfSplitSpam = trainDFSpam.text.str.split(expand=True).stack().value_counts().to_frame()

trainDFHam = trainDF[trainDF["spam"]=="ham"]
trainDFHam["text"]=trainDFHam["text"].str.lower()
dfSplitHam = trainDFHam.text.str.split(expand=True).stack().value_counts().to_frame()

total = dfSplitSpam.join(dfSplitHam,lsuffix='_spam', rsuffix='_ham')
total.fillna(0.001,inplace=True)
spamTotal=total["count_spam"].sum()
hamTotal = total["count_ham"].sum()
total["vraisemblance"]=(total["count_spam"]/spamTotal)/(total["count_ham"]/hamTotal)

spamProbality = spamTotal/hamTotal

C:\Users\melos\AppData\Local\Temp\ipykernel_11020\3630862678.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\melos\AppData\Local\Temp\ipykernel_11020\3630862678.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [22]:
total

,count_spam,count_ham,vraisemblance
to,344,755.000,1.029901
a,201,544.000,0.835182
call,172,99.000,3.927146
you,146,842.000,0.391944
your,131,199.000,1.487997
...,...,...,...
presence,1,0.001,2260.392172
responding,1,2.000,1.130196
everyone,1,3.000,0.753464
romcapspam,1,0.001,2260.392172


In [23]:
df = total.sort_values("vraisemblance", ascending=False).head(40)

In [24]:
from matplotlib.axis import YAxis
import plotly.express as px
import numpy as np
violin = total[(total["vraisemblance"] > 0) & (total["vraisemblance"] < 10)]
px.violin(violin, y="vraisemblance")

In [25]:
total["vraisemblance"].min()

np.float64(0.022603921719955903)

In [26]:
def TestPhraseList(phraseList,spamProba=1):
    for word in phraseList:
        vraisemblance = total["vraisemblance"].get(word,"NotFound")
        if vraisemblance!="NotFound":
            spamProba*=vraisemblance
    return(spamProba)
print(TestPhraseList(["hello","call","to","win","your","prize"]))
print(TestPhraseList(["hello","call","me","i","miss","you"]))

def TestPhrase(phrase:str,spamProba=1):
    phraseList=phrase.split()
    return TestPhraseList(phraseList,spamProba=spamProba)


print(TestPhrase("Hi how are you ? I have a wonderful proposition for you ! Gain your wonderful price by clicking on this link"))


1222093.1801111624
0.00015832933143104019
82.50063930921645


In [33]:
testDF["Finalresult"] = testDF["text"].apply(lambda x: "spam" if TestPhrase(x) > 1 else "ham")

In [29]:
testDF

,spam,text,Finalresult
0,ham,"Go until jurong point, crazy.. Available only ...",1.264493e-05
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1.019424e+08
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",2.805685e+16
13,ham,I've been searching for the right words to tha...,4.284671e-09
14,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,1.000000e+00
...,...,...,...
5564,ham,Why don't you wait 'til at least wednesday to ...,2.059719e-05
5567,spam,This is the 2nd time we have tried 2 contact u...,1.328633e+25
5568,ham,Will ü b going to esplanade fr home?,2.215640e-02
5569,ham,"Pity, * was in mood for that. So...any other s...",2.167125e-02


In [35]:
wrong = len(testDF[testDF["Finalresult"]!=testDF["spam"]].index)
FP = len(testDF[(testDF["Finalresult"]=="spam")&(testDF["spam"]=="ham")].index)
FN = len(testDF[(testDF["Finalresult"]=="ham")&(testDF["spam"]=="spam")].index)
VP = len(testDF[(testDF["Finalresult"]=="ham")&(testDF["spam"]=="ham")].index)
VN = len(testDF[(testDF["Finalresult"]=="spam")&(testDF["spam"]=="spam")].index)
wrong/len(testDF.index)
exactitude = (VN+VP)/(VN+FN+FP+VP)
print(f"Exactitude : {exactitude:.2f}")

rappel = VP /(VP+FN)
print(f"Rappel : {rappel:.2f}")

print(f"Rappel négative: {VN/(VN+FP):.2f}")

precision = VP/(VP+FP)
print(f"Precision : {precision:.2f}")

print(f"Précision négative: {VN/(VN+FN):.2f}")

F1Score = (2 * precision * rappel) / (precision + rappel)
print(f"F1-Score : {F1Score:.2f}")

print(f"F1-Score Négatif: {(2 * VN/(VN+FN) * VN/(VN+FP))/ (VN/(VN+FN) + VN/(VN+FP)):.2f}")


Exactitude : 0.92
Rappel : 0.99
Rappel négative: 0.65
Precision : 0.92
Précision négative: 0.93
F1-Score : 0.95
F1-Score Négatif: 0.76
